# Bločne šifre

Cilji laboratorijske vaje so sledeči:
- Spoznati kriptografsko-varne generatorje naključnih vrednosti
- Namestiti in se delno spoznati s knjižnico `cryptography`
- Uporabiti bločno šifro AES v števčnem načinu z naključnim IV
- Spoznati, da simetrične šifre ne dajo nikakršnih zagotovil glede celovitosti in izvesti napad
- Uporabiti bločno šifro AES v načinu veriženja tajnopisnih blokov z naključnim IV

## Kriptografsko-varni generatorji naključnih vrednosti

Vsak generator naključnih vrednosti ni primeren za uporabo v kriptografiji. Tisti, ki so, se imenujejo (kriptografsko-)varni generatorji.

Naključne vrednosti bomo v Pythonu ustvarili s pomočjo modula `secrets`. [Povezava na dokumentacijo.](https://docs.python.org/3/library/secrets.html)

In [80]:
import secrets

Npr. pridobimo 16 bajtov za simetrični ključ ali za vrednost IV.

In [81]:
key = secrets.token_bytes(16)
iv = secrets.token_bytes(16)

print(f"Key: {key.hex()}")
print(f"IV: {iv.hex()}")

Key: 86ada9e98a4e03fb7f1371df4d85e4f2
IV: 00dfd47094899587e2a34241722530cf


Alternativno lahko uporabimo tudi funkcijo `os.urandom(int)`, ki vrne naključni niz bajtov želene dolžine.

In [82]:
import os

key = os.urandom(16)
iv = os.urandom(16)

print(f"Key: {key.hex()}")
print(f"IV: {iv.hex()}")

Key: 3e3945fd9152ad83a8de5c54bfe342da
IV: 72474070fd117612a8e8f8d194c488d8


Oba načina sta primerna za ustvarjanje kriptografsko-varnih naključnih vrednosti in bi morala delovati v vseh operacijskih sistemih.

## Knijižnica `cryptography`

Pri predmetu bomo v Pythonu uporabljali kriptografsko knjižnico [`cryptography`; povezava do dokumentacije.](https://cryptography.io/en/latest) Namestimo jo z ukazom preko ukaznega poziva.

```
$ pip install cryptography
```

V grobem je knjižnica razdeljena v dva dela:

- varni (enostavni) del, ki od razvijalca ne terja veliko konfiguracijskih nastavitev in je primeren tudi za kriptografsko manj vešče uporabnike, in
- nevarni (hazmat) del, kjer se pričakuje, da veste, kaj počnete.

Prvi ponuja omejen nabor funkcionalnosti, je lažji za uporabo in v njem težje napravimo traparije. Drugi je veliko bolj zmogljiv, a bolj zahteven za uporabo in v kolikor ne veste, kaj počnete, lahko hitro naredite napako. Te so v kriptografiji drage, saj se običajno ugotovijo šele po uspešno izvedenem napadu.

Če menite, da bomo uporabljali enostavni del, se motite. Namen predmeta je, da se izučite za pravilno uporabo _nevarnega_. Znanje bo seveda prenosljivo tudi v druge programske jezike in knjižnice, saj je pogojeno z razumevanjem delovanja kriptografskih gradnikov.

## Naloga 1: Šifriranje z AES-CTR

Začnimo z najbolj enostavnim primerom v kodi: šifriranje AES-CTR, kjer je IV naključno izbran. [Povezava do dokumentacije:](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#) pri kriptografiji je __res nujno__, da poznamo dokumentacijo in ne programiramo po 'občutku'. Zato __le odprite__ podano povezavo in __pred vsako uporabo preberite dokumentacijo__. 

Najprej uvozimo zahtevane pakete.

In [83]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

Za šifriranje potrebujemo
- sporočilo v čistopisu
- ključ
- IV
- šifro -- algoritem $E$

In [84]:
# sporočilo -- ne pozabimo ga postrojiti
msg = "Dober dan!".encode("utf8")

# ključ
k = os.urandom(16)

# IV
iv = os.urandom(16)

# algoritem (ta del je odvisen od knjižnice): pri knjižnici cryptography
# je treba narediti objekt Cipher in v njem izbrati šifrirni algoritem ter način delovanja
cipher = Cipher(
    algorithms.AES(k), # algoritem in ključ
    modes.CTR(iv)) # način delovanja in IV
encryptor = cipher.encryptor() # izberemo E(ncryptor) ali D(ecryptor)

Šifriranje je sedaj _enostavno_:

1. Vmesnik je zastavljen, da lahko šifriramo večkrat, npr. da nalagamo podatke iz več datotek in nimamo vsega v pomnilniku.
2. Ko pokličemo `encryptor.update(pt)`, se podan čistopis šifrira in nastali tajnopis vrne, a ne nujno ves: lahko se zgodi, da zadnji blok čistopisa ne sovpada z velikostjo bloka bločne šifre. V tem primeru se zadnji blok čistopisa še nahaja nešifriran v objektu `encryptor` in čaka morebitne nove podatke, da se blok zapolni.
3. Šifriranje zaključimo s klicem funkcije `encryptor.finalize()` -- takrat se ves morebitni čistopis v objektu `encryptor` šifrira in vrne.

Vse rezultate klicev funkcije `update(pt)` ter na koncu tudi klica `finalize()` dodajamo na seznam bajtov, ki predstavlja končni tajnopis. K temu moramo še dodati IV, sicer ne bo mogoče dešifrirati.

In [85]:
# V našem primeru je celoten čistopis v spremenljivki msg zato potrebujemo le en klic
# metode update in zaključek s finalize
ct = encryptor.update(msg) + encryptor.finalize()

print(f"Ključ: {k.hex()}")
print(f"IV: {iv.hex()}")
print(f"Dolžina čistopisa {len(msg)}, čistopis: {msg.hex()}")
print(f"Dolžina tajnopisa {len(ct)}, tajnopis: {ct.hex()}")

Ključ: 1b24192a827dd4e73cecfbf80cacbf2e
IV: 3c5aabbd35599580ef566ec25636607d
Dolžina čistopisa 10, čistopis: 446f6265722064616e21
Dolžina tajnopisa 10, tajnopis: 0df28298b12696851ff9


Dešifriranje je zelo podobno, potrebujemo:

- Tajnopis
- Ključ
- IV, ki je bil uporabljen pri šifriranju
- Šifro -- algoritem $D$

In [86]:
# spremenljivke k, iv, ct (od zgoraj) zaporedoma predstavljajo ključ, IV in tajnopis

# algoritem dešifriranja
decryptor = Cipher(algorithms.AES(k), modes.CTR(iv)).decryptor()
pt = decryptor.update(ct) + decryptor.finalize()
print("Dešifriran tajnopis:", pt.decode("utf8"))

Dešifriran tajnopis: Dober dan!


Implementirajte funkcijo `enc_aes_ctr(k, pt, iv=None)` skladno z navodili, podanimi v komentarju.

In [87]:
def enc_aes_ctr(k: bytes, pt: bytes, iv: bytes=None) -> tuple[bytes, bytes]:
    if not iv:
        iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(k), modes.CTR(iv)).encryptor()
    """Šifrira čistopis `pt` s ključem `k`. 
    .
    Če je `iv` podan (tj. ni nastavljen na None), ga uporabi. Če ni, ustvari novega in uporabi tega.
    
    Vrne par (IV, tajnopis)."""
    
    ct = cipher.update(pt) + cipher.finalize()
    print(iv, ct)
    return iv, ct
    

Enotski test.

In [88]:
def test_enc_aes_ctr():   
    key = bytes.fromhex("6141fb1142cd0611dd95798fd95352bb")
    iv = bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    
    msg = "Hello World!".encode("utf8")
    iv, ct = enc_aes_ctr(key, msg, iv)

    assert iv == bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    assert ct == bytes.fromhex("aa9169004da9689efa74f00d")

test_enc_aes_ctr()

b'\xde\xdc\xa9t/\xa8a<mB\xea\xbd\xean\xdb\xa7' b'\xaa\x91i\x00M\xa9h\x9e\xfat\xf0\r'


Implementirajte funkcijo `dec_aes_ctr(k, pt, iv)` skladno z navodili, podanimi v komentarju.

In [89]:
def dec_aes_ctr(k: bytes, ct: bytes, iv: bytes) -> bytes:
    decript = Cipher(algorithms.AES(k), modes.CTR(iv)).decryptor()
    pt = decript.update(ct) + decript.finalize()
    """Dešifrira tajnopis s ključem `k` in podanim `IV`.
    
    Vrne čistopis."""
    return pt

In [90]:
def test_dec_aes_ctr():  
    key = bytes.fromhex("6141fb1142cd0611dd95798fd95352bb")
    iv = bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    ct = bytes.fromhex("aa9169004da9689efa74f00d")
    
    pt = dec_aes_ctr(key, ct, iv)

    assert pt == b"Hello World!"

test_dec_aes_ctr()

## Naloga 2: Šifriranje vsebine

Preden se lotite te naloge, si poglejte aplikacijo telefonski imenik v datoteki `pb.py` ter njeno modifikacijo v datoteki `pb_solved.py`. Prva različica shranjuje podatke v datoteko `data/phonebook.json` v čistopisu, druga pa vsebino pred shranjevanjem na disk šifrira. Ko vam bo aplikacija razumljiva, se lotite naloge 2.

Uporabite funkciji iz prejšnje naloge in šifrirajte vsebino datoteke `data/ic-za-narodov-blagor.txt`.

Vsebino nato shranite v datoteko `data/ic-za-narodov-blagor.txt.enc`. 

Ne pozabite: Poleg samega tajnopisa, morate v datoteko shraniti še IV. Dokaj standarden način je, da IV in nastali tajnopis konkatenirate in skupaj shranite v datoteko. (Ločevanje je potem preprosto: prvih 16 bajtov v datoteki je IV, preostali pa so dejanski tajnopis.)

In [91]:
def read_and_encrypt_file(key: bytes, filename: str):
    iv = os. urandom(16)
    
    with open(filename, 'br') as h:
            pt = h.read()
            
    cifer = Cipher(algorithms.AES(key), modes.CTR(iv)).encryptor()
    ct = cifer.update(pt) + cifer.finalize()
    
    with open(filename + ".enc", 'wb') as h:
        h.write(iv)
        h.write(ct)
    
    
    """Prebere vsebino datoteke `filename`, jo šifrira z AES-CTR in ključem key ter zapiše nazaj disk. 
    
    Ime šifrirane datoteke je enako kot ime nešifrirane, le da se ji še doda še končnica `.enc`,
    npr `data/ic-za-narodov-blagor.txt` se naj šifrira v  `data/ic-za-narodov-blagor.txt.enc`
    
    Funkcija ne vrača rezultata."""

Naslednja funkcija naj prebere datoteko `filename` (npr. `data/ic-za-narodov-blagor.txt.enc`), vsebino razčleni v IV in tajnopis, slednjega dešifrira in ga vrne kot rezultat.

In [92]:
def read_and_decrypt(key: bytes, filename: str) -> bytes:
    """Prebere vsebino datoteke `filename`, jo dešifrira in vrne čistopis."""
    with open(filename, 'br') as h:
            data = h.read()
        
    iv, ct = data[:16], data[16:]
    cifer_d = Cipher(algorithms.AES(key), modes.CTR(iv)).decryptor()
    
    pt = cifer_d.update(ct) + cifer_d.finalize()
    return pt

Spodnji test preveri, ali sta implementaciji zgornjih funkcij pravilni: 
- najprej šifrira datoteko `data/ic-za-narodov-blagor.txt` s funkcijo `read_and_encrypt_file`,
- nato s funkcijo `read_and_decrypt` prebere tajnopis in ga dešifrira,
- ter na koncu preveri ali je dešifrirana vsebina pravilna.

In [93]:
def test_enc_dec_file(key):
    # šifriramo in shranimo na disk
    read_and_encrypt_file(key, "data/ic-za-narodov-blagor.txt")
    
    # preberemo in dešifriramo 
    pt = read_and_decrypt(key, "data/ic-za-narodov-blagor.txt.enc")
    
    with open("data/ic-za-narodov-blagor.txt", "rb") as h:
        data = h.read()
        
    assert data == pt

test_enc_dec_file(os.urandom(16))

## Naloga 3: Gnetljivost tajnopisa

Ne pozabite: AES-CTR je na koncu tokovna šifra -- četudi je narejen iz psevdonaključne permutacije: ustvarimo (psevdo)naključno podlogo in jo z operacijo XOR združimo s čistopisom.

Imamo enako nalogo kot pred dvema tednoma. Za osvežitev spomina.

---

Kot napadalec bomo spremenili tajnopis, tako da bo sprememba vidna v čistopisu. Zgodba je sledeča.

Ana želi poslati zaupno pošto Boru, vas, ki igrate vlogo napadalca Nandija, pa vsebina sporočila _zares_ zanima.

Na srečo vam gre nekaj reči na roko. 

Prvič, Anin računalnik nima internetne povezave, vaš mobilni telefon pa. Zato ji prijazno ponudite, da zanjo postavite mobilno dostopno točko, preko katere se bo lahko povezala v internet in dostopala do poštnega strežnika. Poštni strežnik bo nato sporočilo dostavil Boru. 

Ana in Bor uporabljata poseben protokol za pošto: protokol FMTP -- Funny Mail Transfer Protocol.  Gre za preprost besedilno-osnovan protokol: prva vrstica označuje naslov prejemnika, druga naslov pošiljatelja, tretja zadevo, četrta je vedno prazna in na koncu je sporočilo.

Vse, kar mora Anin poštni odjemalec storiti, da pošlje sporočilo, je, da strežniku FMTP dostavi besedilni niz, podoben naslednjemu.

```txt
prejemnik@enadomena.com
posiljatelj@drugadomena.com
Zadeva sporocila

<Samo sporocilo>
```

Vse morebitne predhodne ali zaključne presledke v vsaki vrstici poštni strežnik ignorira oz. odstrani pred obdelavo. Na primer, e-pošto zgoraj bi lahko napisali kot spodnjo in ne bi bilo nobene razlike.


```txt
                    prejemnik@enadomena.com               
    posiljatelj@drugadomena.com                    
Zadeva sporocila

<Samo sporocilo>
```

Tretjič, Ana vam v dobri veri -- kakšna naivnost! -- pove, da pošilja elektronsko sporočilo Boru in da uporablja protokol FMTP. (Torej poznate strukturo sporočila in vsebino prve vrstice čistopisa.)

__Tokrat Ana uporablja AES v načinu CTR z naključnim IV.__ Mehanizmov za zagotavljanje celovitosti pravtako ni.

Ker nastopate v vlogi **posrednika**, morate sedaj spremeniti sporočilo tako, da strežnik FMTP ne bo poslal pošte Boru, temveč jo bo poslal na naslov, ki ga nadzirate vi, npr. `nandi@obvlada.si`.

---

(Namig: Če ste nalogo rešili prejšnjikrat, boste tokrat imeli toliko dela, kolikor traja kopiranje rešitve. Sicer vzemite nalogo kot ponoven poskus reševanja.)

In [94]:
# Ana in Bor si delita skrivni ključ dolžine 16 bajtov
ana_bor_psk = os.urandom(16)

# Ana pripravi sporočilo
email = """bor@student.uni-lj.si
ana@student.uni-lj.si
Hej

Na faksu si pozabil kapo.""".strip()

# In ga šifrira z AES-CTR z naključnim IV
iv, ct = enc_aes_ctr(ana_bor_psk, email.encode("utf8"))

# Nato ga pošlje (to simuliramo) na streznik FMTP
print("Ana pošilja šifrirano sporočilo na strežnik")
print("Tajnopis (HEX):", ct.hex())
print("IV (HEX):", iv.hex())

b'\x9c\xf8J\xa0\xd0\xf1y\xe4j\x1a,jF\x13\xe0\xe6' b'jK\x89P\xcf\xac\x1aS\xb4;\xf5_\xef\xaf\x03\x1aj\x93\x93\xf5\x88\xcf\x8b\xd9\x91)\xb7X\xda\x81\xb4\xa4"\x80\xcct(\x93\xb8\xdd@\xacb\xe0\x08Wc\xbf\x857\xef\r\x08\xf7\x89\x98\xe1\x81W\'\xa2d\x96\xa3\x80\xa2\x9b3DI\x9e-2\xf0'
Ana pošilja šifrirano sporočilo na strežnik
Tajnopis (HEX): 6a4b8950cfac1a53b43bf55fefaf031a6a9393f588cf8bd99129b758da81b4a42280cc742893b8dd40ac62e0085763bf8537ef0d08f78998e1815727a26496a380a29b3344499e2d32f0
IV (HEX): 9cf84aa0d0f179e46a1a2c6a4613e0e6


Sedaj je na potezi napadalec Nandi.

Implementirajte funckijo `change_ct(ct, new_email)`, ki na vhodu prejme tajnopis in email naslov. Spremenite tajnopis tako, da ga bo strežnik FMTP še vedno brez dešifriral, a kot prejemnik ne bo naveden Bor, temveč email naslov, ki je podan v argumentu `new_email`. Predpostavite lahko, da bo `new_email` vedno krajši ali enak od naslova `bor@student.uni-lj.si`.

In [95]:
def change_ct(ct, new_email):
    b_new_email = new_email.encode("utf8")
    
    c = bytearray(ct)
    for i in range(len(b_new_email)):
        c[i] = ct[i] ^ (ord(email[i]) ^ b_new_email[i])
    return bytes(c)
    
# nandi@obvlada.si




S poganjanjem spodnje celice lahko preverite, ali vaša funkcija deluje pravilno. 

In [96]:
def fmtp_receive(key, ct, iv):
    print("Streznik FMTP prejel sporocilo:", ct.hex())
    pt = dec_aes_ctr(key, ct, iv)
    print("Desifrirano sporocilo:")
    print(pt.decode("utf8"))

fmtp_receive(ana_bor_psk, change_ct(ct, "nandi@obvlada.si"), iv)

Streznik FMTP prejel sporocilo: 66459574d5980055a739e015fbef195e6a9393f588cf8bd99129b758da81b4a42280cc742893b8dd40ac62e0085763bf8537ef0d08f78998e1815727a26496a380a29b3344499e2d32f0
Desifrirano sporocilo:
nandi@obvlada.silj.si
ana@student.uni-lj.si
Hej

Na faksu si pozabil kapo.


## Naloga 4: Šifriranje z AES-CBC

Sedaj bomo ponovili zgodbo iz naloge 1, le da bomo tokrat uporabili AES v načinu veriženja tajnopisnih blokov z naključnim IV. 

Gre za način, ki je varen, a ima slabše karakteristike kot CTR, zaradi česar se počasi opušča. Kljub temu ga kot varnostni strokovnjak moramo poznati.

Implementirajte funkcijo `enc_aes_cbc(k, pt, iv=None)` skladno z navodili podanimi v komentarju.

Pozor: Način CBC zahteva, da čistopis pred šifriranjem podložite (angl. padding). Po dešifriranju pa je podlogo potrebno odstraniti. V knjižnici `cryptography` morate to storiti sami, a si pri tem lahko pomagate z modulom [symmetric padding.](https://cryptography.io/en/latest/hazmat/primitives/padding/)

Privzeto podlagamo standardom PKCS#7. Sledi primer.

In [97]:
from cryptography.hazmat.primitives import padding

padder = padding.PKCS7(128).padder() # podlagamo do velikost 128 bitov
padded_data = padder.update(b"Tole bosta dva bloka podatkov.") # naš vhodni podatek
print(f"Po klicu update: {len(padded_data)}: {padded_data}")

padded_data += padder.finalize() # zaključimo podlaganje
print(f"Po klicu finalize: {len(padded_data)}: {padded_data}")

Po klicu update: 30: b'Tole bosta dva bloka podatkov.'
Po klicu finalize: 32: b'Tole bosta dva bloka podatkov.\x02\x02'


Če želimo podlogo odstraniti uporabimo `unpadder`.

In [98]:
unpadder = padding.PKCS7(128).unpadder() # podloženo je za velikst bloka 128 bitov
unpadded_data = unpadder.update(padded_data) # naložimo podložene podatke
print(f"Po klicu update: {len(unpadded_data)}: {unpadded_data}") 

unpadded_data += unpadder.finalize() # zaključimo
print(f"Po klicu finalize: {len(unpadded_data)}: {unpadded_data}")

Po klicu update: 16: b'Tole bosta dva b'
Po klicu finalize: 30: b'Tole bosta dva bloka podatkov.'


Morebitne dodatne informacije poiščite v dokumentaciji.

In [99]:
def enc_aes_cbc(k: bytes, pt: bytes, iv: bytes) -> tuple[bytes, bytes]:
    if not iv:
        iv = os.urandom(16)
     
    padder = padding.PKCS7(128).padder()
    padded = padder.update(pt) + padder.finalize()

    cipher = Cipher(algorithms.AES(k), modes.CBC(iv)).encryptor()
    ct = cipher.update(padded) + cipher.finalize()
           
    return iv, ct
    """Šifrira čistopis s ključem v načinu AES-CBC.
    
    Če IV ni podan, ustvari naključnega.
    Kot rezultat vrne par (IV, tajnopis)."""

In [100]:
def test_enc_aes_cbc():   
    key = bytes.fromhex("6141fb1142cd0611dd95798fd95352bb")
    iv = bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    
    msg = "Hello World!".encode("utf8")
    iv, ct = enc_aes_cbc(key, msg, iv)
    
    assert iv == bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    assert ct == bytes.fromhex("1fc519b6a47b3c562d905965e6e2f465")

test_enc_aes_cbc()

Implementirajte še pripadajočo funkcijo za dešifriranje. 

Zgodba s podlaganjem je tu obrnjena: po dešifriranju je treba podlogo odstraniti.

In [101]:
def dec_aes_cbc(k: bytes, ct: bytes, iv: bytes) -> bytes:
    cipher = Cipher(algorithms.AES(k), modes.CBC(iv)).decryptor()
    ct_padded = cipher.update(ct) + cipher.finalize()
    
    unpadder = padding.PKCS7(128).unpadder()
    pt = unpadder.update(ct_padded) + unpadder.finalize()
    return pt
    """Dešifrira tajnopis `ct` s ključem `k` in vrednostjo `IV`. 
    
    Vrne čistopis."""

In [102]:
def test_dec_aes_cbc():  
    key = bytes.fromhex("6141fb1142cd0611dd95798fd95352bb")
    iv = bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    ct = bytes.fromhex("1fc519b6a47b3c562d905965e6e2f465")
    
    pt = dec_aes_cbc(key, ct, iv)

    assert pt == b"Hello World!"

test_dec_aes_cbc()